# Tidy-Data

In [18]:
# libraries
import numpy as np
import pandas as pd
import altair as alt
# warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# display settings
pd.options.display.max_columns = None

### Raw data

**First, convert the raw .xlsx file to a .csv file and then use '.read_csv'**

In [19]:
# Because NaN is a float, this forces an array of integers with any missing values (i.e. Year) to become floating point (2008 to 2008.0)
# Use 'Int64' while reading in the .csv file 
col_types = {
    'reportyear': 'Int64',
    'race_eth_code': 'Int64',
    'geotypevalue': 'Int64',
    'county_fips': 'Int64',
    'region_code': 'Int64',
    'pop_trans_acc': 'Int64',
    'pop2010': 'Int64'
}

# reading in the data set
data = pd.read_csv('data/walkable_distance_to_public_transportation.csv', 
                   dtype = col_types,
                   low_memory = False)

# print
data.head(3)

,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,version,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
0,51,Percent of population residing within ½ mile o...,2008,3,AfricanAm,CO,6061,Placer,Placer,6061,Sacramento Area,8,55,4427,0.012424,0.009161,0.015687,0.001665,13.399974,NaN,0.070460,26Nov2013,NaN,NaN,NaN,NaN,NaN,NaN
1,51,Percent of population residing within ½ mile o...,2008,1,AIAN,CO,6061,Placer,Placer,6061,Sacramento Area,8,51,2080,0.024519,0.017873,0.031166,0.003391,13.830066,NaN,0.139059,26Nov2013,NaN,NaN,NaN,NaN,NaN,NaN
2,51,Percent of population residing within ½ mile o...,2008,2,Asian,CO,6061,Placer,Placer,6061,Sacramento Area,8,117,19963,0.005861,0.004802,0.006920,0.000540,9.217872,NaN,0.033239,26Nov2013,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning the data

**data_mod1:** removing the last row

In [20]:
# Last row is the 'END OF TABLE' row with only 'NaN' values
data_mod1 = data.copy()
data_mod1.tail(1)

,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,version,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
66009,END OF TABLE,NaN,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Last row should now have index '66008'
data_mod1 = data_mod1[:-1]
data_mod1.tail(1)

,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,version,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
66008,51,Percent of population within 1/2 mile of majo...,2012,9,Total,RE,14,Southern California,NaN,<NA>,Southern California,14,6350660,18051534,0.351807,0.351587,0.352028,0.000112,0.031948,NaN,NaN,##############################################...,NaN,NaN,NaN,NaN,NaN,1.000021


**data_mod2:** removing columns that repeat the same value

In [22]:
data_mod2 = data_mod1.copy()

# Each value is the same for 'ind_id' and 'ind_definition' columns
print(data_mod2['ind_id'].unique())
print(data_mod2['ind_definition'].unique())

['51' 'END OF TABLE']
['Percent of population residing within ½ mile of a major transit stop'
 nan ' Percent of population within 1/2 mile of major transit stop']


In [23]:
# Dropping the columns
data_mod2 = data_mod2.drop(columns = {'ind_id', 'ind_definition'})
data_mod2.head(1)

,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,version,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
0,2008,3,AfricanAm,CO,6061,Placer,Placer,6061,Sacramento Area,8,55,4427,0.012424,0.009161,0.015687,0.001665,13.399974,NaN,0.07046,26Nov2013,NaN,NaN,NaN,NaN,NaN,NaN


**data_mod3**: examining the 'version' column

In [24]:
data_mod3 = data_mod2.copy()
data_mod3['version']

0                                                26Nov2013
1                                                26Nov2013
2                                                26Nov2013
3                                                26Nov2013
4                                                26Nov2013
                               ...                        
66004    ##############################################...
66005    ##############################################...
66006    ##############################################...
66007    ##############################################...
66008    ##############################################...
Name: version, Length: 66009, dtype: object

_The 'version ' column contains a mix of 'DD/M/YYYY' & '######...' values. Remove the version column, as we already have Year values._

In [25]:
data_mod3 = data_mod3.drop(columns = {'version'})

**data_mod4**: renaming the 'reportyear' column to 'year'

In [26]:
data_mod4 = data_mod3.copy()

data_mod4 = data_mod4.rename(
    columns = {'reportyear': 'year',}
)

data_mod4.head(1)

,year,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
0,2008,3,AfricanAm,CO,6061,Placer,Placer,6061,Sacramento Area,8,55,4427,0.012424,0.009161,0.015687,0.001665,13.399974,NaN,0.07046,NaN,NaN,NaN,NaN,NaN,NaN


**data_mod5**: removing the rows with Year absent

In [27]:
data_mod5 = data_mod4.copy()

missing_values = data_mod5.isna().sum()
missing_values

year                 3
race_eth_code        3
race_eth_name        3
geotype              3
geotypevalue         3
geoname             57
county_name         66
county_fips         66
region_name          3
region_code          3
pop_trans_acc     1185
pop2010              3
p_trans_acc       1560
LL_95CI           1185
UL_95CI           1560
se                1185
rse              14703
SAC_decile       65496
SAC_RR           61434
MTC_decile       64208
MTC_RR           49926
SD_decile        65329
SD_RR            59942
SC_decile        61716
SC_RR            28285
dtype: int64

There are **3 rows** where the variable Year has missing values. There are other variables also missing 3 times, alongside Year, such as "race_eth_code" and "race_eth_name."

Examining the rows where Year is missing:

In [28]:
data_mod5[data_mod5['year'].isna()]

,year,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
4662,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21034,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27173,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


All of the values are missing in these rows. Since the data set is very large, we can remove these 3 rows, as they are <0.01% of the overall data.

In [29]:
data_mod5 = data_mod5.dropna(subset=['year'])

In [30]:
# Checking that there are no missing values for 'year'
data_mod5['year'].isna().sum()

0

**data_mod6:** removing variables missing more than 80% of the time

In [31]:
data_mod6 = data_mod5.copy()

# what variables are missing more than 40% of the time
(data_mod6.isna().sum()) / len(data_mod6) > 0.40

year             False
race_eth_code    False
race_eth_name    False
geotype          False
geotypevalue     False
geoname          False
county_name      False
county_fips      False
region_name      False
region_code      False
pop_trans_acc    False
pop2010          False
p_trans_acc      False
LL_95CI          False
UL_95CI          False
se               False
rse              False
SAC_decile        True
SAC_RR            True
MTC_decile        True
MTC_RR            True
SD_decile         True
SD_RR             True
SC_decile         True
SC_RR             True
dtype: bool

These are comparison statistics ('SAC_decile' to 'SC_RR'), while the others ('LL_95CI' to 'rse') are statistical reliability calculations.

In [32]:
# dropping the columns
data_mod6 = data_mod6.drop(columns = {'SAC_decile','SAC_RR', 'MTC_decile', 'SD_decile', 'SD_RR', 'SC_decile', 'MTC_RR', 'SC_RR'})

data_mod6.head(-1)

,year,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse
0,2008,3,AfricanAm,CO,6061,Placer,Placer,6061,Sacramento Area,8,55,4427,0.012424,0.009161,0.015687,0.001665,13.399974
1,2008,1,AIAN,CO,6061,Placer,Placer,6061,Sacramento Area,8,51,2080,0.024519,0.017873,0.031166,0.003391,13.830066
2,2008,2,Asian,CO,6061,Placer,Placer,6061,Sacramento Area,8,117,19963,0.005861,0.004802,0.006920,0.000540,9.217872
3,2008,4,Latino,CO,6061,Placer,Placer,6061,Sacramento Area,8,1835,44710,0.041042,0.039203,0.042881,0.000938,2.286029
4,2008,7,Multiple,CO,6061,Placer,Placer,6061,Sacramento Area,8,241,10658,0.022612,0.019790,0.025435,0.001440,6.368321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66003,2012,4,Latino,RE,14,Southern California,NaN,<NA>,Southern California,14,3324031,8169102,0.406903,0.406566,0.407240,0.000172,0.042241
66004,2012,5,NHOPI,RE,14,Southern California,NaN,<NA>,Southern California,14,14864,43955,0.338164,0.333741,0.342587,0.002256,0.667279
66005,2012,6,White,RE,14,Southern California,NaN,<NA>,Southern California,14,1490980,6028281,0.247331,0.246986,0.247675,0.000176,0.071050
66006,2012,7,Multiple,RE,14,Southern California,NaN,<NA>,Southern California,14,116421,378200,0.307829,0.306358,0.309300,0.000751,0.243832


### Exporting to 'tidy-data'

**Exporting to a .csv file**

In [33]:
data_mod6.to_csv('tidy-data')